In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
train = pd.read_csv("train.csv")

In [9]:
train.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
449,450,1,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.5,C104,S
706,707,1,2,"Kelly, Mrs. Florence ""Fannie""",female,45.0,0,0,223596,13.5,NaN,S


In [11]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
len(train)

891

In [15]:
train.Age.fillna(train.Age.mean(),inplace = True)

In [16]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [17]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [20]:
train.Name.nunique() == len(train.Name)

True

In [21]:
train['children'] = train.Age.apply(lambda x: 1 if x < 15 else 0)

In [22]:
train.sample(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,children
207,208,1,3,"Albimona, Mr. Nassef Cassem",male,26.000000,0,0,2699,18.7875,NaN,C,0
589,590,0,3,"Murdlin, Mr. Joseph",male,29.699118,0,0,A./5. 3235,8.0500,NaN,S,0
629,630,0,3,"O'Connell, Mr. Patrick D",male,29.699118,0,0,334912,7.7333,NaN,Q,0
631,632,0,3,"Lundahl, Mr. Johan Svensson",male,51.000000,0,0,347743,7.0542,NaN,S,0


In [23]:
train = pd.get_dummies(train, columns=['Sex','Pclass'])

In [25]:
train.sample(4)

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,children,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
737,738,1,"Lesurer, Mr. Gustave J",35.0,0,0,PC 17755,512.3292,B101,C,0,0,1,1,0,0
723,724,0,"Hodges, Mr. Henry Price",50.0,0,0,250643,13.0000,NaN,S,0,0,1,0,1,0
858,859,1,"Baclini, Mrs. Solomon (Latifa Qurban)",24.0,0,3,2666,19.2583,NaN,C,0,1,0,0,0,1
267,268,1,"Persson, Mr. Ernst Ulrik",25.0,1,0,347083,7.7750,NaN,S,0,0,1,0,0,1


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [27]:
train.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'children', 'Sex_female', 'Sex_male',
       'Pclass_1', 'Pclass_2', 'Pclass_3'],
      dtype='object')

In [28]:
y_columns = ['Survived']

x_columns = ['Age', 'SibSp', 'Parch', 'Fare',
             'children', 'Sex_female','Pclass_1', 'Pclass_2']

x_train,x_val, y_train, y_val = train_test_split(train[x_columns], 
                                                 train[y_columns],
                                                 test_size=0.2,
                                                 random_state=42)

In [30]:
model = RandomForestClassifier(n_estimators=100,
                                 max_depth=5, 
                                 random_state=42,
                                 oob_score = True,
                                criterion= 'entropy'
                              )
clf = model.fit(x_train,np.ravel(y_train))

In [31]:
clf.score(x_val,y_val)

0.7988826815642458

In [32]:
print(pd.crosstab(np.array(y_val).reshape(1,-1)[0], 
                  clf.predict(x_val) ,
                  rownames=['Actual'], 
                  colnames=['Predicted'],
                  margins=True))

Predicted    0   1  All
Actual                 
0           92  13  105
1           23  51   74
All        115  64  179


In [33]:
cross_val_score(estimator=clf, 
                X=(x_train),
                y= y_train.values.ravel(), 
                cv=3,
                n_jobs=-1,
                verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.0s finished


array([ 0.81932773,  0.83122363,  0.79324895])

In [34]:

from sklearn.feature_selection import RFECV

selector = RFECV(clf, step=1, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y_train)
print(pd.DataFrame(selector.ranking_, index = x_columns, columns=['importance']).sort_values('importance', ascending=True))


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


            importance
Age                  1
SibSp                1
Parch                1
Fare                 1
children             1
Sex_female           1
Pclass_1             1
Pclass_2             1


In [35]:

from sklearn.model_selection import GridSearchCV
a  = GridSearchCV(clf, {'n_estimators':[100,200,300]}, cv = 4)
# a.fit(x_train, np.array(y_train).reshape(1,712)[0])
# a.fit(x_train, np.array(y_train).reshape(1,-1)[0])
a.fit(x_train, np.array(np.ravel(y_train)).reshape(1,-1)[0])


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=True, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [37]:
a.best_params_

{'n_estimators': 300}